## 04 - Alternative Architecture

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers


### 4.1 - Bridging Section

In the previous sections we gave a deep exploration the changes in performance of a MLP on the KDD under the action of three different optimisers: 'adam', 'NAG' and 'SGD'. Alas, in more typical cyber settings we could only dream of having such a clean dataset, and this is where we find motivation for the coming section. When faced with the challenge of trying to find a more typical cyber datasets we were forced to abstract slightly- instead of asking if we could find a realistic cyber dataset, we asked what structures cyber data often fall into, and concluded that sequence (or time series) data, and graphs were common structures. We eventually found a dataset which could kill two birds with one stone, a dataset of malware (and non-malware) API call sequences [1]. From this we could just use the sequence data immediately, as well as trivially generate graphs for each sequence, where we connected API call `a` and API call `b` if and only if `b` was directly after `a` at some point in the sequence.

We start by reading in the API call dataset.

In [2]:
df = pd.read_csv("API_Calls.csv")
df.head()

,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,malware
0,071e8c3f8922e186e57548cd4c703a5d,112,274,158,215,274,158,215,298,76,...,71,297,135,171,215,35,208,56,71,1
1,33f8e6d08a6aae939f25a8e0d63dd523,82,208,187,208,172,117,172,117,172,...,81,240,117,71,297,135,171,215,35,1
2,b68abd064e975e1c6d5f25e748663076,16,110,240,117,240,117,240,117,240,...,65,112,123,65,112,123,65,113,112,1
3,72049be7bd30ea61297ea624ae198067,82,208,187,208,172,117,172,117,172,...,208,302,208,302,187,208,302,228,302,1
4,c9b3700a77facf29172f32df6bc77f48,82,240,117,240,117,240,117,240,117,...,209,260,40,209,260,141,260,141,260,1


We see that the first 100 calls are given in sequence for each program, and the final column gives us a binary label, whether or not the sequence is malware. We also note that each different kind of API call is given by an integer. It's clear that we don't need to worry about the `hash` column, and immediately drop this.

In [3]:
df = df.drop('hash', 1)

/Users/billnunn/opt/miniconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


We think it's worth noting that the reader may correctly identify that our API call dataset is at least as idealised and unrealistic as the KDD data. To this we say that this section isn't really about the classification performance: the API data is instead used as a 'toy' example to explore neural architectures more appropriate for realistic cyber settings. The first architecture we try out is in **Section 4.3**, we construct a recurrent neural network (RNN) on the API calls treated as sequence data. After this we then discuss convolutional networks in **Section 4.4**, paying special attention to the case where our input data is graphs, as is the case for our API call graphs.

### 4.2 - RNN Data Preperation

As is always the case, we do some quick preprocessing before we can get on with our RNNs in **Section 4.3**. We first decide upon what proportion of the data we want to use for training.


In [4]:
train_test_ratio = 0.7
cut = round(df.shape[0]*0.7)

Great, we shuffle `df` and split into the training and test sets. We also carry out a quick sanity check to see that the full data has been partitioned into training and test sets.

In [5]:
df = df.sample(frac=1)
train_df = df[0:cut]
test_df = df[cut:df.shape[0]+1]
train_df.shape[0] + test_df.shape[0] == df.shape[0]

True

Finally we need to convert our dataframes into `numpy` arrays, ready for input into a `keras` model.

In [6]:
X_train = np.array(train_df.drop('malware', 1))
X_test = np.array(test_df.drop('malware', 1))

y_train = np.array(train_df['malware'])
y_test = np.array(test_df['malware'])

/Users/billnunn/opt/miniconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/Users/billnunn/opt/miniconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


We are good to continue.

### 4.3 - Recurrent Neural Networks

We encountered the underpinning mathematical ideas of recurrent neural networks in the DST lectures. We therefore refrain from discussing the mathematical background of this architecture (luckily/unluckily we are barely in control of our mathematical spewings and some maths will undoubtedly emerge in **Section 4.4**). As with all our previous networks we start by defining the layers of our network.

In [26]:
model = keras.Sequential()

# Input layer, note input dimension is dictionary size + 1:
model.add(layers.Embedding(input_dim=307, output_dim=20))

# RNN layer with 12 internal units:
model.add(layers.LSTM(12))

# Dense layer with 12 units:
model.add(layers.Dense(12, activation='relu'))

# Bianry classification layer, sigmoid to emulate a probability:
model.add(layers.Dense(1, activation='sigmoid'))

And to summarise this model:

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 20)          6140      
_________________________________________________________________
lstm (LSTM)                  (None, 12)                1584      
_________________________________________________________________
dense (Dense)                (None, 12)                156       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 13        
Total params: 7,893
Trainable params: 7,893
Non-trainable params: 0
_________________________________________________________________


Our plan from here is to continue the analysis of the earlier parts of the report, and compile using a range of optimisers and then fit these models to our sequence data. When we compile we ensure we use binary cross entropy as our loss function- remember we are only doing binary classification with this dataset.

Before compiling our model we note that the dataset has highly imbalanced classes.

In [9]:
mal = 0
good = 0
for i in df['malware']:
    if i==0:
        good = good + 1
    else:
        mal = mal + 1

print("Number of malware sequences:", mal, 
      ". Number of non-malware sequences:", good)

Number of malware sequences: 42797 . Number of non-malware sequences: 1079


As we shall see, the class imbalance is a real issue- our go to loss functions (binary cross entropy and friends) may not be the most sensible thing to want to minimse. We found a neat blog post [5] and we would have given this a try if we had the time.

#### 4.3.1 - Adam Optimiser

In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=[tf.keras.metrics.TrueNegatives(),
                       tf.keras.metrics.TruePositives()])

Because of the imbalanced classes, we use the number of true negatives and true positives as our metrics, and compare to the number in the training set:

In [11]:
trainmal = 0
traingood = 0
for i in train_df['malware']:
    if i==0:
        traingood = traingood + 1
    else:
        trainmal = trainmal + 1
print("Number of training non-malware sequences:", traingood, 
      ". Number of training malware sequences:", trainmal)

Number of training non-malware sequences: 740 . Number of training malware sequences: 29973


With the above numbers in mind we train our network with the adam optimiser.

In [12]:
model.fit(X_train, y_train, batch_size=100, epochs=10)

Epoch 1/10
308/308 [==============================] - 53s 173ms/step - loss: 0.1425 - true_negatives: 0.0000e+00 - true_positives: 29973.0000
Epoch 2/10
308/308 [==============================] - 49s 159ms/step - loss: 0.0779 - true_negatives: 0.0000e+00 - true_positives: 29973.00005s - loss: 0.0786 - true_negatives: 0.0
Epoch 3/10
308/308 [==============================] - 53s 171ms/step - loss: 0.0676 - true_negatives: 0.0000e+00 - true_positives: 29973.0000
Epoch 4/10
308/308 [==============================] - 52s 169ms/step - loss: 0.0604 - true_negatives: 1.0000 - true_positives: 29973.0000
Epoch 5/10
308/308 [==============================] - 52s 169ms/step - loss: 0.0562 - true_negatives: 150.0000 - true_positives: 29954.0000
Epoch 6/10
308/308 [==============================] - 55s 178ms/step - loss: 0.0528 - true_negatives: 271.0000 - true_positives: 29921.00002s - loss: 0.0529 - true_negatives: 253.0000 - true_posi
Epoch 7/10
308/308 [==============================] - 49s 160

We now evaluate our network on the test set. Again the true negative and true positive counts are what we're interested in:

In [13]:
testmal = 0
testgood = 0
for i in test_df['malware']:
    if i==0:
        testgood = testgood + 1
    else:
        testmal = testmal + 1
print("Number of testing non-malware sequences:", testgood, 
      ". Number of testing malware sequences:", testmal)

Number of testing non-malware sequences: 339 . Number of testing malware sequences: 12824


In [17]:
model.evaluate(X_test, y_test)

412/412 [==============================] - 12s 28ms/step - loss: 0.0560 - true_negatives: 174.0000 - true_positives: 12798.0000


[0.055981628596782684, 174.0, 12798.0]

We see that the loss on the test set holds up well to the training loss, and it is therefore unlikely that we've overfitted. As was forewarned we have a greatly reduced accuracy on the smaller class, we've done significant playing behind the scenes with continued use of binary cross entropy as our loss but can't seem to resolve this. When we begin to capture a significantly better proportion of true negatives we have already begun an inescapable flirtation with overfitting. To combat this we could try adding drop out, or the alternative loss function suggested in [5].

#### 4.3.2 - NAG Optimiser

We pinch Mo's code for defining a NAG optimiser.

In [15]:
nag_opt = tf.keras.optimizers.SGD(momentum = 0.9, nesterov = True)

From here we repeat the previous section exchanging `adam` for `nag_opt`.

In [19]:
model = keras.Sequential()

# Input layer, note input dimension is dictionary size + 1:
model.add(layers.Embedding(input_dim=307, output_dim=20))

# RNN layer with 12 internal units:
model.add(layers.LSTM(12))

# Dense layer with 12 units:
model.add(layers.Dense(12, activation='relu'))

# Bianry classification layer, sigmoid to emulate a probability:
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=nag_opt, 
              metrics=[tf.keras.metrics.TrueNegatives(),
                       tf.keras.metrics.TruePositives()])

In [20]:
model.fit(X_train, y_train, batch_size=100, epochs=10)

Epoch 1/10
308/308 [==============================] - 54s 175ms/step - loss: 0.1399 - true_negatives_1: 1.0000 - true_positives_1: 29965.0000
Epoch 2/10
308/308 [==============================] - 56s 182ms/step - loss: 0.1132 - true_negatives_1: 0.0000e+00 - true_positives_1: 29973.0000
Epoch 3/10
308/308 [==============================] - 58s 187ms/step - loss: 0.1128 - true_negatives_1: 0.0000e+00 - true_positives_1: 29973.0000
Epoch 4/10
308/308 [==============================] - 50s 163ms/step - loss: 0.1123 - true_negatives_1: 0.0000e+00 - true_positives_1: 29973.0000
Epoch 5/10
308/308 [==============================] - 48s 156ms/step - loss: 0.1112 - true_negatives_1: 0.0000e+00 - true_positives_1: 29973.0000
Epoch 6/10
308/308 [==============================] - 49s 160ms/step - loss: 0.1081 - true_negatives_1: 0.0000e+00 - true_positives_1: 29973.0000
Epoch 7/10
308/308 [==============================] - 52s 167ms/step - loss: 0.1000 - true_negatives_1: 0.0000e+00 - true_positi

In [21]:
model.evaluate(X_test, y_test)

412/412 [==============================] - 12s 28ms/step - loss: 0.0912 - true_negatives_1: 0.0000e+00 - true_positives_1: 12824.0000


[0.09121925383806229, 0.0, 12824.0]

Interesting, the outcome of the NAG is brutal (and happens repeatedly upon re-runs): every call sequence is deemed malware. Like adam did initially, NAG gets caught in a position where it predicts no true negatives on the training set, and gets stuck at the 'local minima' produced by the large class imbalance. In contrast, the adam optimiser somwhow escapes by epoch four- this really feels like black magic and we must learn out how it works in detail at a later date! Furthermore, we wonder if there is something inherent in the LSTM architecture which results in such a large disparity between adam and NAG. Recall that with the MLP architecture of the earlier sections the contrast wasn't quite so stark

#### 4.3.3 - SGD Optimiser

Again we thank Mo for his earlier code, by nicking his other optimiser. We don't think things bode well for SGD after NAG's poor showing.

In [16]:
sgd_opt = tf.keras.optimizers.SGD(momentum = 0.9, nesterov = False)

From here you have the pleasure of more repeat code, third time's a charm.

In [23]:
model = keras.Sequential()

# Input layer, note input dimension is dictionary size + 1:
model.add(layers.Embedding(input_dim=307, output_dim=20))

# RNN layer with 12 internal units:
model.add(layers.LSTM(12))

# Dense layer with 12 units:
model.add(layers.Dense(12, activation='relu'))

# Bianry classification layer, sigmoid to emulate a probability:
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=sgd_opt, 
              metrics=[tf.keras.metrics.TrueNegatives(),
                       tf.keras.metrics.TruePositives()])

In [24]:
model.fit(X_train, y_train, batch_size=100, epochs=10)

Epoch 1/10
308/308 [==============================] - 53s 172ms/step - loss: 0.1508 - true_negatives_2: 1.0000 - true_positives_2: 29885.0000
Epoch 2/10
308/308 [==============================] - 51s 165ms/step - loss: 0.1133 - true_negatives_2: 0.0000e+00 - true_positives_2: 29973.0000
Epoch 3/10
308/308 [==============================] - 50s 163ms/step - loss: 0.1130 - true_negatives_2: 0.0000e+00 - true_positives_2: 29973.0000
Epoch 4/10
308/308 [==============================] - 50s 162ms/step - loss: 0.1126 - true_negatives_2: 0.0000e+00 - true_positives_2: 29973.0000
Epoch 5/10
308/308 [==============================] - 49s 159ms/step - loss: 0.1121 - true_negatives_2: 0.0000e+00 - true_positives_2: 29973.0000
Epoch 6/10
308/308 [==============================] - 50s 163ms/step - loss: 0.1114 - true_negatives_2: 0.0000e+00 - true_positives_2: 29973.0000
Epoch 7/10
308/308 [==============================] - 49s 158ms/step - loss: 0.1096 - true_negatives_2: 0.0000e+00 - true_positi

In [25]:
model.evaluate(X_test, y_test)

412/412 [==============================] - 12s 30ms/step - loss: 0.1024 - true_negatives_2: 0.0000e+00 - true_positives_2: 12824.0000


[0.10241524130105972, 0.0, 12824.0]

As we anticipated, SGD should do no better than NAG.

### 4.4 - Graph Convolution Neural Networks

Unlike recurrent neural networks, we didn't encounter graph convolution in DST lectures and so we start this section by giving some of the mathematical background on the subject. Our discussion closely follows the graph convolution method detailed in [2].

The method detailed in [2] is surprising: it acts on graphs directly; the graphs aren’t required to have the same number of nodes; the nodes themselves don’t even need to be labeled. Furthermore, the whole network can be fitted with backpropagation alone. Earlier neural network methods for graph classification (such as PATCHY-SAN [3]) were reliant on heavy preprocessing of data, and the preprocessing could not be learned via direct backpropagation of the final output. The suggestion for why such an improbable sounding method is effective is that we implicitly apply a graph kernel similar to a Weisfeiler-Lehman kernel [4] (see Bill's reflection for discussion of WL kernels).

#### 4.4.1 - Stages of the Architecture

The network splits into three main parts:

* The convolution layers.
* The sort pooling layer.
* The remaining layers.

We explain only the first two of these parts as the 'remaining layers' don't take a specific form.

#### 4.4.2 - The Convolution Layers

Let $G$ be a graph with $n$ vertices. 

Let's suppose we choose to have $h$ convolution layers indexed by $t$, which output an $n$ rowed matrix $Z^t$, where we want the $i$ th rows of our $Z$ matrices to give information about the 'structural role' of vertex $i$. We define $Z^0$ to be a special matrix $X$ which has dimension $n$ by $c_0$. In the case where we have labeled vertices we can let $X$ be the one hot encoding of the vertices i.e. the $n$ by $n$ identity matrix. In the case where the nodes are unlabeled we can also make a sensible choice for $X$, the column vector of normalised node degrees is suggested.

From here the convolution step takes the following form:

$$Z^{t+1} = f(\tilde D^{-1} \tilde A Z^{t}W^{t})$$

Where $W$ is the $c_t$ by $c_{t+1}$ matrix whose columns are the filter weights (in the usual CNN sense) for a given filter. The matrix $\tilde A$ is the $A+I$ where $A$ is the adjacency matrix, and $\tilde D$ is the degree matrix of the extended adjacency matrix $\tilde A$. We note that the application of $\tilde D^{-1}$ is to normalise the result of applying $\tilde A$. Finally $f$ is our non-linear activation function applied to the matrix point wise. Since $Z^0$ is $n$ by $c_0$ and $W^0$ is $c_0$ by $c_1$ it follows that $Z^1$ is $n$ by $c_1$. And similarly $Z^t$ is $n$ by $c_t$ using induction. Now matrix $Z_t$ really just stores the outcome of applying $c_t$ convolutional filters on the outputs of the previous time step's filters, where only filter outputs of adjacent nodes are mixed in accordance with $\tilde A$.

As is usual in CNNs the outputs of the $h$ convolution layers are gathered up- we simply column bind matrices $Z^1, Z^2,..., Z^h$ to get matrix $Z^{1:h}$ which is clearly $n$ by $\sum^{h}_{t=1} c_t$. Each row of $Z^{1:h}$ contains deep structural information about its corresponding node, and hopefully useful information once we've trained the filter weights! The following lemma (of ours) illustrates this fact somewhat- realising it helped the above methodology *click* into place.

**Lemma:** Let $G$ be a graph with a non-trivial graph automorphism. The rows of $Z^{1:h}$ corresponding to automorphic nodes are identical.

This observation leads us nicely onto the pooling step of our network.

#### 4.4.3 - The Sort Pool Layer

In the case where the nodes don't have a natural order (i.e they're unlabeled) we use the structural insight the convolution steps have granted us to order the nodes in a consistent way before passing to the next step of the network, we call this process sort pooling.

If necessary we add empty rows to $Z^{1:h}$ until it has $k$ rows, so that all every graph's $Z^{1:h}$ matrix is the same size. We order each row by right to left lexicographical ordering and call the resulting matrix $Z^{sp}$ and we've finished the sort pooling. In order to backpropagate through the convolution layers we only need to keep track of how the rows were re-ordered.

#### 4.4.X - The Unfinished

As is probably obvious, we intended to build and fit graph convolution networks using the three different optimisers. We didn't have the hours to do so and will have to name our old intentions 'unfinished business'. Like Clint Eastwood maybe

### Sources for Section 4

[1] https://www.kaggle.com/ang3loliveira/malware-analysis-datasets-api-call-sequences

[2] Zhang, Cui, Neumann, and Chen, An End-to-End Deep Learning Architecture for Graph Classification. AAAI-18. 2018

[3] Niepert, Ahmed, and Kutzkov, Learning convolutional neural networks for graphs. In Proceedings of the 33rd annual international conference on machine learning. 2016.

[4] Shervashidze, Schweitzer, Leeuwen, Mehlhorn, Borgwardt. Weisfeiler-Lehman graph kernels. Journal of Machine Learning Research, 12:25392561. 2011.

[5] https://towardsdatascience.com/a-loss-function-suitable-for-class-imbalanced-data-focal-loss-af1702d75d75 